Work in progress

In [36]:
#importeer libraries
import pandas as pd
import geopandas as gpd
import branca
import folium
import mapclassify
import matplotlib
import ipywidgets as widgets
from IPython.display import display
import numpy as np


In [2]:
#Ecologie delfland
data_2024 = pd.read_excel("..\Ecologie_tot_2024.xlsx")
data_macrofyten_aanvullend_richness = pd.read_excel("..\Macrofyten_Hackathon_update25_06.xlsx", sheet_name = "Macrofyten_Richness")
data_macrofyten_aanvullend_EKR = pd.read_excel("..\Macrofyten_Hackathon_update25_06.xlsx", sheet_name = "Macrofyten_EKR")
data_macrofyten_aanvullend_Chemie = pd.read_excel("..\Macrofyten_Hackathon_update25_06.xlsx", sheet_name = "Chemie_meetgegevens")
data_macrofyten_aanvullend_Meetpunt = pd.read_excel("..\Macrofyten_Hackathon_update25_06.xlsx", sheet_name = "Meetpunt_informatie")
data_macrofyten_aanvullend_Weer = pd.read_excel("..\Macrofyten_Hackathon_update25_06.xlsx", sheet_name = "Weer_historie")

In [3]:
#Make ecologie regional plot
data_macrofyten_aanvullend_Meetpunt_gpd = gpd.GeoDataFrame(
    data_macrofyten_aanvullend_Meetpunt , geometry=gpd.points_from_xy(
        data_macrofyten_aanvullend_Meetpunt.X, 
        data_macrofyten_aanvullend_Meetpunt.Y), 
        crs="EPSG:28992"
)
data_macrofyten_aanvullend_Meetpunt_gpd = data_macrofyten_aanvullend_Meetpunt_gpd.to_crs("EPSG:4326")


In [4]:
#join meetobject met ruwe meetdata
data_macrofyten_Meetpunt_ruwe_data_gpd = data_macrofyten_aanvullend_Meetpunt_gpd.merge( data_2024, on = "MeetObject")


In [5]:
data_macrofyten_Meetpunt_ruwe_data_gpd["DateTime"] = pd.to_datetime(data_macrofyten_Meetpunt_ruwe_data_gpd.CollectieDatumTijd, format = "%d-%m-%y%y %H:%M")
data_macrofyten_Meetpunt_ruwe_data_gpd["Jaar"] = data_macrofyten_Meetpunt_ruwe_data_gpd["DateTime"].dt.to_period('Y')
data_macrofyten_Meetpunt_ruwe_data_gpd["Maand"] = data_macrofyten_Meetpunt_ruwe_data_gpd["DateTime"].dt.to_period('M')
print(data_macrofyten_Meetpunt_ruwe_data_gpd["Jaar"])

0        2019
1        2019
2        2019
3        2019
4        2019
         ... 
25888    2013
25889    2013
25890    2013
25891    2013
25892    2013
Name: Jaar, Length: 25893, dtype: period[A-DEC]


In [39]:
#Group by species and see where occurs
data_macrofyten_Meetpunt_ruwe_data_pd = pd.DataFrame(data_macrofyten_Meetpunt_ruwe_data_gpd).sort_values(by=["Parameter","MeetObject","Jaar"])
subset_data_pd = data_macrofyten_Meetpunt_ruwe_data_pd[["Parameter","MeetObject","Jaar","WaardeGemeten"]]
grouped_data = subset_data_pd[
    subset_data_pd.WaardeGemeten > 0].groupby(
        ["Parameter","MeetObject"]).apply(lambda x: x["WaardeGemeten"].pct_change().mean()).reset_index(name='avg_change').fillna(0)
grouped_data['category'] = np.where(grouped_data['avg_change'] < -0.1, 'decreasing', np.where(grouped_data['avg_change'] > 0.1, 'increasing', 'no change'))

In [40]:
#Join data
data_grouped_gpd = data_macrofyten_aanvullend_Meetpunt_gpd.merge( grouped_data , on = "MeetObject")
data_grouped_gpd.head()


,MeetObject,Naam,Gemeente,X,Y,Meetnet,Landgebruik,Vaarwater,Gebied_Type,geometry,Parameter,avg_change,category
0,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,EBEO_H&W,Stad,Ja,Boezem,POINT (4.34334 52.02050),Acorus calamus,0.0,no change
1,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,EBEO_H&W,Stad,Ja,Boezem,POINT (4.34334 52.02050),Alnus glutinosa,0.0,no change
2,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,EBEO_H&W,Stad,Ja,Boezem,POINT (4.34334 52.02050),Artemisia vulgaris,0.0,no change
3,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,EBEO_H&W,Stad,Ja,Boezem,POINT (4.34334 52.02050),Bidens frondosa,0.0,no change
4,OW001-000,Kerstanje_ spoorbrug,Delft,83350,448531,EBEO_H&W,Stad,Ja,Boezem,POINT (4.34334 52.02050),Carex hirta,0.0,no change


In [ ]:
grouped_data = subset_data_pd[
    subset_data_pd.WaardeGemeten > 0].groupby(
        ["Parameter","MeetObject"]).apply(lambda x: x["WaardeGemeten"].pct_change().mean()).reset_index(name='avg_change').fillna(0)
grouped_data['category'] = np.where(grouped_data['avg_change'] < -0.1, 'decreasing', np.where(grouped_data['avg_change'] > 0.1, 'increasing', 'no change'))


grouped_data_decreasing = grouped_data.loc[grouped_data.category == "decreasing",["MeetObject", "Parameter"]].groupby(["MeetObject"]).count().reset_index()
grouped_data_increasing = grouped_data.loc[grouped_data.category == "increasing",["MeetObject", "Parameter"]].groupby(["MeetObject"]).count().reset_index()
grouped_data_decreasing.head()
grouped_data_increasing.head()

,MeetObject,Parameter
0,OW004-000,9
1,OW004-002,8
2,OW005-000,8
3,OW006-016,6
4,OW007-000,4


In [53]:
data_grouped_gpd_decreasing = data_macrofyten_aanvullend_Meetpunt_gpd.merge(grouped_data_decreasing , on = "MeetObject")
data_grouped_gpd_decreasing.explore()

In [54]:
data_grouped_gpd_increasing = data_macrofyten_aanvullend_Meetpunt_gpd.merge(grouped_data_increasing , on = "MeetObject")
data_grouped_gpd_increasing.explore()

In [10]:
#print(data_macrofyten_aanvullend)
data_macrofyten_aanvullend.head()

,MeetObject,Jaar,Maand,Opnemer,Richness_Aantallen
0,OW005-000,2020,8,RTR,2
1,OW007-000,2022,8,EFL,1
2,OW007-001,2020,8,RTR,7
3,OW014-000,2022,8,EFL,1
4,OW015-003,2019,8,EXT,10
